In [5]:
## Create train, dev, test splits
import pandas as pd
from sklearn.model_selection import train_test_split



In [6]:

file_path = '250k_rndm_zinc_drugs_clean_3.csv'

df = pd.read_csv(file_path)

train_data, temp_data = train_test_split(df, test_size=0.2, random_state=42)

dev_data, test_data = train_test_split(temp_data, test_size= 0.5, random_state=42)

train_data.to_csv('zinc-250k-train.csv', index = False)
dev_data.to_csv('zinc-250k-val.csv', index = False)
test_data.to_csv('zinc-250k-test.csv', index = False)

In [ ]:
########## DEBUG FROM HERE ON ################

In [3]:
from transformers import AutoTokenizer 

In [4]:

tokenizer_name='seyonec/ChemBERTa-zinc-base-v1'
tok = AutoTokenizer.from_pretrained(tokenizer_name)

tokenizer_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

c:\Users\yasmi\miniconda3\envs\cs224n\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yasmi\.cache\huggingface\hub\models--seyonec--ChemBERTa-zinc-base-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/501 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/9.43k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/3.21k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [7]:
import torch

In [18]:
class SmilesDataset(Dataset):
    def __init__(self, csv_file, tokenizer_name='seyonec/ChemBERTa-zinc-base-v1', max_length=80):
        self.data = pd.read_csv(csv_file)
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Smile strings and scores
        smiles = self.data.iloc[idx]['smiles']
        logp = self.data.iloc[idx]['logP']
        qed = self.data.iloc[idx]['qed']
        sas = self.data.iloc[idx]['SAS']

        print('smiles', smiles)

        # Tokenize smiles string

        tokens = self.tokenizer(smiles, padding = 'max_length', max_length = self.max_length, truncation = True, return_tensors = 'pt')

        input_ids = tokens['input_ids'].squeeze(0) # Remove batch dim

        logp_tensor = torch.tensor(logp, dtype = torch.float32)
        qed_tensor = torch.tensor(qed, dtype = torch.float32)
        sas_tensor = torch.tensor(sas, dtype = torch.float32)
       
       

        return {
            'input_ids': input_ids,
            'logp': logp_tensor,
            'qed': qed_tensor,
            'sas': sas_tensor
        }


In [5]:
from torch.utils.data import Dataset, DataLoader

In [10]:
import pandas as pd

In [19]:
train_dataset =SmilesDataset('C:\\Users\\yasmi\\OneDrive\\My Documents\\Stanford 2024-2025\\CS230\\cs230\\data\\zinc-250k-train.csv')
 
print('Created datasets')

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8,
                                )

Created datasets


In [20]:
train_dataset[0]

smiles COC(=O)c1ccc(CN(C)Cc2nc(N)c3c(C)c(C)sc3n2)cc1



{'input_ids': tensor([  0, 307, 263,  51,  13,  71,  21, 264,  12, 286,  12,  39,  13, 267,
          22, 270,  12,  50,  13,  71,  23,  71,  12,  39,  13,  71,  12,  39,
          13, 296,  23,  82,  22,  13, 261,  21, 203,   2,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1]),
 'logp': tensor(3.3089),
 'qed': tensor(0.6819),
 'sas': tensor(2.3377)}

In [ ]:
for batch in (train_dataloader):

    input_ids = batch['input_ids']

    print(input_ids)

       


smiles CCc1ccccc1NC(=O)Cn1c(=O)n(Cc2ccc(Cl)cc2)c(=O)c2ccccc21

smiles CC1(C)CC(=O)C2=C(C1)O[C@@H](N)[C@@H](C#N)[C@@H]2C1=c2ccccc2=[NH+]C1

smiles Cc1sc(NC(=O)CN2C(=O)N[C@]3(CCCC[C@H]3C)C2=O)nc1-c1ccc(Br)cc1

smiles CCC(=O)OC[C@@H]1CCC[NH2+]C1

smiles CC[C@H](C)NC(=O)[C@H]1CCCCN1S(=O)(=O)c1ccc(F)cc1

smiles CC(=O)Nc1ccc(S(=O)(=O)N2CCN=C2SCc2cccc(C(F)(F)F)c2)cc1

smiles Cc1cccc(NC(=O)CSc2nnc3c(Cl)cc(Cl)cn23)c1

smiles COc1cc(OC)c(NCc2cnn(-c3ccccc3)c2)cc1F

dict_keys(['input_ids', 'logp', 'qed', 'sas'])
smiles COc1ccc([C@@H](C)NC(=O)Cc2cccc(C)c2)cc1

smiles CC(=O)N[C@H](C(=O)Oc1ccc(Oc2nccs2)cc1)C1CCCC1

smiles COCC(=O)Nc1cccc([C@H]2SCC(=O)N2c2ccc(OC)cc2)c1

smiles CC[C@@H](c1nnnn1C(C)(C)C)[NH+](Cc1cc2ccc(C)cc2[nH]c1=O)C1CCCCC1

smiles COc1ccccc1S(=O)(=O)NCCC(=O)N(C)C

smiles N#CCCn1cc(/C=C\C(=O)[C@@H](C#N)c2ccccn2)c2ccccc21

smiles Cc1ccc(NC(=O)CS(=O)(=O)c2nccn2C)c(Br)c1

smiles CC(C)[C@H](NC(=O)[C@H](C)NC(N)=O)c1ccc(C(C)(C)C)cc1

dict_keys(['input_ids', 'logp', 'qed', 'sas'])
smiles COC(

KeyboardInterrupt: 